In [1]:
#importing libraries
import pandas as pd
import numpy as np
from collections import Counter
from collections import defaultdict

from sklearn.feature_selection import VarianceThreshold, SelectKBest, SelectFromModel, RFE
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# data visualization
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
plt.style.use("ggplot")
rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 300)

In [2]:
df = pd.read_csv('data/data_full_upto_varThr.csv') #quello con anche il test set
df_train = df.loc[:1827,:].copy() #solo train set, si lavora su questo per via del bias che si andrebbe a creare
df_train.head()

,vocal_channel,emotion,emotional_intensity,statement,repetition,sex,frame_count,sum,skew,lag1_kur,zc_sum,mfcc_sum,mfcc_mean,mfcc_max,mfcc_q05,mfcc_q25,mfcc_q50,mfcc_q75,mfcc_q95,mfcc_q99,sc_sum,sc_mean,sc_std,sc_min,sc_max,sc_q01,sc_q05,sc_q25,sc_q50,sc_q75,sc_q95,sc_q99,sc_kur,stft_sum,sum_w1,kur_w1,skew_w1,lag1_kur_w1,lag1_skew_w1,zc_sum_w1,zc_q95_w1,mfcc_sum_w1,mfcc_mean_w1,mfcc_std_w1,mfcc_max_w1,mfcc_q05_w1,mfcc_q25_w1,mfcc_q50_w1,mfcc_q75_w1,mfcc_q95_w1,mfcc_q99_w1,sc_sum_w1,sc_std_w1,sc_min_w1,sc_q25_w1,sc_q75_w1,sc_q95_w1,sc_kur_w1,sc_skew_w1,stft_sum_w1,stft_kur_w1,sum_w2,kur_w2,zc_sum_w2,mfcc_min_w2,mfcc_max_w2,mfcc_q75_w2,mfcc_q95_w2,mfcc_kur_w2,sc_sum_w2,sc_mean_w2,sc_std_w2,sc_min_w2,sc_max_w2,sc_q05_w2,sc_q25_w2,sc_q50_w2,sc_kur_w2,stft_sum_w2,sum_w3,kur_w3,lag1_kur_w3,zc_sum_w3,mfcc_mean_w3,mfcc_q05_w3,mfcc_q95_w3,mfcc_q99_w3,mfcc_kur_w3,sc_sum_w3,sc_mean_w3,sc_std_w3,sc_min_w3,sc_max_w3,sc_q25_w3,sc_kur_w3,stft_sum_w3,sum_w4,kur_w4,skew_w4,lag1_kur_w4,lag1_skew_w4,zc_sum_w4,zc_q95_w4,mfcc_sum_w4,mfcc_mean_w4,mfcc_std_w4,mfcc_max_w4,mfcc_q25_w4,mfcc_q50_w4,mfcc_q95_w4,mfcc_q99_w4,sc_sum_w4,sc_mean_w4,sc_std_w4,sc_min_w4,sc_max_w4,sc_q25_w4,sc_q75_w4,sc_kur_w4,sc_skew_w4,stft_sum_w4,stft_kur_w4,stft_skew_w4
0,speech,neutral,normal,Kids are talking by the door,1st,M,0.106792,0.506722,0.784505,0.260775,0.129522,0.591481,0.423199,0.717587,0.635723,0.896695,0.479183,0.261387,0.447466,0.752627,0.655516,0.954851,0.882301,0.000000,0.705391,0.385894,0.451166,0.560393,0.927656,0.951307,0.950086,0.655447,0.014217,0.528794,0.552984,0.572236,0.618010,0.699286,1.000000,0.062556,0.0,0.445355,0.133570,0.933467,0.265192,0.374045,0.679413,0.329009,0.167593,0.191928,0.483503,0.807990,0.092930,0.975799,0.965340,0.999875,0.935376,0.692039,0.780737,0.853646,0.503824,0.553195,0.691411,0.224991,0.299631,0.628919,0.652833,0.509086,0.438077,0.210342,0.405315,0.593581,0.393472,0.528451,0.404438,0.385595,0.378612,0.087534,0.466630,0.564012,0.590970,0.486622,0.224149,0.591696,0.801812,0.374269,0.833184,0.481993,0.307510,0.365604,0.769611,0.318640,0.578369,0.291006,0.533887,0.553313,0.482849,0.290004,0.738540,0.108887,0.593791,0.448805,1.0,0.597521,0.216581,0.866067,0.159312,0.653604,0.488662,0.281894,0.187912,0.738302,0.964313,0.268862,0.000000,0.901624,0.936516,0.976689,0.958815,0.102331,0.514047,0.988635,0.048224
1,speech,neutral,normal,Kids are talking by the door,2nd,M,0.116504,0.503248,0.807974,0.225783,0.148238,0.592678,0.436033,0.598435,0.506675,0.862466,0.479183,0.278588,0.470939,0.729264,0.624961,0.901829,0.812829,0.401039,0.450510,0.467991,0.450760,0.561798,0.799551,0.926962,0.943297,0.633036,0.013852,0.533221,0.600963,0.435323,0.230835,0.467376,0.617102,0.149082,1.0,0.573211,0.331577,0.819130,0.135787,0.516668,0.679413,0.522225,0.469625,0.271325,0.489737,0.764826,0.161923,0.628552,0.909069,0.989784,0.930195,0.638155,0.389996,0.867947,0.192102,0.451680,0.699709,0.198031,0.290148,0.550263,0.603532,0.441019,0.457699,0.226723,0.420190,0.495440,0.525674,0.367284,0.493723,0.394864,0.414946,0.051492,0.519341,0.611536,0.543484,0.539834,0.235869,0.614794,0.852725,0.521718,0.839149,0.449225,0.255697,0.303490,0.658337,0.305612,0.450865,0.293301,0.552003,0.507090,0.510608,0.339563,0.407132,0.128112,0.592731,0.396224,0.0,0.555675,0.159624,0.898993,0.209908,0.632088,0.413982,0.253659,0.267796,0.742450,0.956018,0.268475,0.869244,0.900040,0.872086,0.990271,0.177813,0.526318,0.483277,0.095336,0.360443
2,speech,neutral,normal,Dogs are sitting by the door,1st,M,0.097081,0.507227,0.872440,0.269902,0.104711,0.602877,0.432130,0.667726,0.887036,0.896088,0.479183,0.235460,0.380469,0.801084,0.619848,0.921414,0.853955,0.000000,0.459613,0.393383,0.431250,0.472504,0.911578,0.930051,0.947636,0.642306,0.013639,0.495870,0.599825,0.100512,0.413890,0.112541,0.527296,0.149221,1.0,0.602522,0.342404,0.821121,0.156563,0.832058,0.701664,0.558589,0.466631,0.278917,0.491881,0.754973,0.123756,0.799765,0.905290,0.993358,0.930973,0.494276,0.483352,0.846438,0.131463,0.5646

In [3]:
# target variable moved to the end
targ = 'zc_sum' #choose the target variable
categoricals = ['vocal_channel','emotion','emotional_intensity','statement','repetition','sex']
non_targ = categoricals.copy()
try:
    non_targ.remove(targ)
except:
    pass
target_variable = df.pop(targ)
df[targ] = target_variable
print(non_targ)
df.head()

['vocal_channel', 'emotion', 'emotional_intensity', 'statement', 'repetition', 'sex']


C:\Users\User\AppData\Local\Temp\ipykernel_24696\3658534449.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[targ] = target_variable


,vocal_channel,emotion,emotional_intensity,statement,repetition,sex,frame_count,sum,skew,lag1_kur,mfcc_sum,mfcc_mean,mfcc_max,mfcc_q05,mfcc_q25,mfcc_q50,mfcc_q75,mfcc_q95,mfcc_q99,sc_sum,sc_mean,sc_std,sc_min,sc_max,sc_q01,sc_q05,sc_q25,sc_q50,sc_q75,sc_q95,sc_q99,sc_kur,stft_sum,sum_w1,kur_w1,skew_w1,lag1_kur_w1,lag1_skew_w1,zc_sum_w1,zc_q95_w1,mfcc_sum_w1,mfcc_mean_w1,mfcc_std_w1,mfcc_max_w1,mfcc_q05_w1,mfcc_q25_w1,mfcc_q50_w1,mfcc_q75_w1,mfcc_q95_w1,mfcc_q99_w1,sc_sum_w1,sc_std_w1,sc_min_w1,sc_q25_w1,sc_q75_w1,sc_q95_w1,sc_kur_w1,sc_skew_w1,stft_sum_w1,stft_kur_w1,sum_w2,kur_w2,zc_sum_w2,mfcc_min_w2,mfcc_max_w2,mfcc_q75_w2,mfcc_q95_w2,mfcc_kur_w2,sc_sum_w2,sc_mean_w2,sc_std_w2,sc_min_w2,sc_max_w2,sc_q05_w2,sc_q25_w2,sc_q50_w2,sc_kur_w2,stft_sum_w2,sum_w3,kur_w3,lag1_kur_w3,zc_sum_w3,mfcc_mean_w3,mfcc_q05_w3,mfcc_q95_w3,mfcc_q99_w3,mfcc_kur_w3,sc_sum_w3,sc_mean_w3,sc_std_w3,sc_min_w3,sc_max_w3,sc_q25_w3,sc_kur_w3,stft_sum_w3,sum_w4,kur_w4,skew_w4,lag1_kur_w4,lag1_skew_w4,zc_sum_w4,zc_q95_w4,mfcc_sum_w4,mfcc_mean_w4,mfcc_std_w4,mfcc_max_w4,mfcc_q25_w4,mfcc_q50_w4,mfcc_q95_w4,mfcc_q99_w4,sc_sum_w4,sc_mean_w4,sc_std_w4,sc_min_w4,sc_max_w4,sc_q25_w4,sc_q75_w4,sc_kur_w4,sc_skew_w4,stft_sum_w4,stft_kur_w4,stft_skew_w4,zc_sum
0,speech,neutral,normal,Kids are talking by the door,1st,M,0.106792,0.506722,0.784505,0.260775,0.591481,0.423199,0.717587,0.635723,0.896695,0.479183,0.261387,0.447466,0.752627,0.655516,0.954851,0.882301,0.000000,0.705391,0.385894,0.451166,0.560393,0.927656,0.951307,0.950086,0.655447,0.014217,0.528794,0.552984,0.572236,0.618010,0.699286,1.000000,0.062556,0.0,0.445355,0.133570,0.933467,0.265192,0.374045,0.679413,0.329009,0.167593,0.191928,0.483503,0.807990,0.092930,0.975799,0.965340,0.999875,0.935376,0.692039,0.780737,0.853646,0.503824,0.553195,0.691411,0.224991,0.299631,0.628919,0.652833,0.509086,0.438077,0.210342,0.405315,0.593581,0.393472,0.528451,0.404438,0.385595,0.378612,0.087534,0.466630,0.564012,0.590970,0.486622,0.224149,0.591696,0.801812,0.374269,0.833184,0.481993,0.307510,0.365604,0.769611,0.318640,0.578369,0.291006,0.533887,0.553313,0.482849,0.290004,0.738540,0.108887,0.593791,0.448805,1.0,0.597521,0.216581,0.866067,0.159312,0.653604,0.488662,0.281894,0.187912,0.738302,0.964313,0.268862,0.000000,0.901624,0.936516,0.976689,0.958815,0.102331,0.514047,0.988635,0.048224,0.129522
1,speech,neutral,normal,Kids are talking by the door,2nd,M,0.116504,0.503248,0.807974,0.225783,0.592678,0.436033,0.598435,0.506675,0.862466,0.479183,0.278588,0.470939,0.729264,0.624961,0.901829,0.812829,0.401039,0.450510,0.467991,0.450760,0.561798,0.799551,0.926962,0.943297,0.633036,0.013852,0.533221,0.600963,0.435323,0.230835,0.467376,0.617102,0.149082,1.0,0.573211,0.331577,0.819130,0.135787,0.516668,0.679413,0.522225,0.469625,0.271325,0.489737,0.764826,0.161923,0.628552,0.909069,0.989784,0.930195,0.638155,0.389996,0.867947,0.192102,0.451680,0.699709,0.198031,0.290148,0.550263,0.603532,0.441019,0.457699,0.226723,0.420190,0.495440,0.525674,0.367284,0.493723,0.394864,0.414946,0.051492,0.519341,0.611536,0.543484,0.539834,0.235869,0.614794,0.852725,0.521718,0.839149,0.449225,0.255697,0.303490,0.658337,0.305612,0.450865,0.293301,0.552003,0.507090,0.510608,0.339563,0.407132,0.128112,0.592731,0.396224,0.0,0.555675,0.159624,0.898993,0.209908,0.632088,0.413982,0.253659,0.267796,0.742450,0.956018,0.268475,0.869244,0.900040,0.872086,0.990271,0.177813,0.526318,0.483277,0.095336,0.360443,0.148238
2,speech,neutral,normal,Dogs are sitting by the door,1st,M,0.097081,0.507227,0.872440,0.269902,0.602877,0.432130,0.667726,0.887036,0.896088,0.479183,0.235460,0.380469,0.801084,0.619848,0.921414,0.853955,0.000000,0.459613,0.393383,0.431250,0.472504,0.911578,0.930051,0.947636,0.642306,0.013639,0.495870,0.599825,0.100512,0.413890,0.112541,0.527296,0.149221,1.0,0.602522,0.342404,0.821121,0.156563,0.832058,0.701664,0.558589,0.466631,0.278917,0.491881,0.754973,0.123756,0.799765,0.905290,0.993358,0.930973,0.494276,0.483352,0.846438,0.131463,0.564682,0.6694

In [4]:
# One_Hot_Econder for categorical variables
df = pd.get_dummies(df, columns = non_targ)
count=0
# moving binarized vars to the beginning of the dataframe
for i in df.loc[:,'vocal_channel_song':].keys():
    temp=df.pop(i)
    df.insert(loc=count, column=i, value=temp)
    count=count+1

df.head()

,vocal_channel_song,vocal_channel_speech,emotion_angry,emotion_calm,emotion_disgust,emotion_fearful,emotion_happy,emotion_neutral,emotion_sad,emotion_surprised,emotional_intensity_normal,emotional_intensity_strong,statement_Dogs are sitting by the door,statement_Kids are talking by the door,repetition_1st,repetition_2nd,sex_F,sex_M,frame_count,sum,skew,lag1_kur,mfcc_sum,mfcc_mean,mfcc_max,mfcc_q05,mfcc_q25,mfcc_q50,mfcc_q75,mfcc_q95,mfcc_q99,sc_sum,sc_mean,sc_std,sc_min,sc_max,sc_q01,sc_q05,sc_q25,sc_q50,sc_q75,sc_q95,sc_q99,sc_kur,stft_sum,sum_w1,kur_w1,skew_w1,lag1_kur_w1,lag1_skew_w1,zc_sum_w1,zc_q95_w1,mfcc_sum_w1,mfcc_mean_w1,mfcc_std_w1,mfcc_max_w1,mfcc_q05_w1,mfcc_q25_w1,mfcc_q50_w1,mfcc_q75_w1,mfcc_q95_w1,mfcc_q99_w1,sc_sum_w1,sc_std_w1,sc_min_w1,sc_q25_w1,sc_q75_w1,sc_q95_w1,sc_kur_w1,sc_skew_w1,stft_sum_w1,stft_kur_w1,sum_w2,kur_w2,zc_sum_w2,mfcc_min_w2,mfcc_max_w2,mfcc_q75_w2,mfcc_q95_w2,mfcc_kur_w2,sc_sum_w2,sc_mean_w2,sc_std_w2,sc_min_w2,sc_max_w2,sc_q05_w2,sc_q25_w2,sc_q50_w2,sc_kur_w2,stft_sum_w2,sum_w3,kur_w3,lag1_kur_w3,zc_sum_w3,mfcc_mean_w3,mfcc_q05_w3,mfcc_q95_w3,mfcc_q99_w3,mfcc_kur_w3,sc_sum_w3,sc_mean_w3,sc_std_w3,sc_min_w3,sc_max_w3,sc_q25_w3,sc_kur_w3,stft_sum_w3,sum_w4,kur_w4,skew_w4,lag1_kur_w4,lag1_skew_w4,zc_sum_w4,zc_q95_w4,mfcc_sum_w4,mfcc_mean_w4,mfcc_std_w4,mfcc_max_w4,mfcc_q25_w4,mfcc_q50_w4,mfcc_q95_w4,mfcc_q99_w4,sc_sum_w4,sc_mean_w4,sc_std_w4,sc_min_w4,sc_max_w4,sc_q25_w4,sc_q75_w4,sc_kur_w4,sc_skew_w4,stft_sum_w4,stft_kur_w4,stft_skew_w4,zc_sum
0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0.106792,0.506722,0.784505,0.260775,0.591481,0.423199,0.717587,0.635723,0.896695,0.479183,0.261387,0.447466,0.752627,0.655516,0.954851,0.882301,0.000000,0.705391,0.385894,0.451166,0.560393,0.927656,0.951307,0.950086,0.655447,0.014217,0.528794,0.552984,0.572236,0.618010,0.699286,1.000000,0.062556,0.0,0.445355,0.133570,0.933467,0.265192,0.374045,0.679413,0.329009,0.167593,0.191928,0.483503,0.807990,0.092930,0.975799,0.965340,0.999875,0.935376,0.692039,0.780737,0.853646,0.503824,0.553195,0.691411,0.224991,0.299631,0.628919,0.652833,0.509086,0.438077,0.210342,0.405315,0.593581,0.393472,0.528451,0.404438,0.385595,0.378612,0.087534,0.466630,0.564012,0.590970,0.486622,0.224149,0.591696,0.801812,0.374269,0.833184,0.481993,0.307510,0.365604,0.769611,0.318640,0.578369,0.291006,0.533887,0.553313,0.482849,0.290004,0.738540,0.108887,0.593791,0.448805,1.0,0.597521,0.216581,0.866067,0.159312,0.653604,0.488662,0.281894,0.187912,0.738302,0.964313,0.268862,0.000000,0.901624,0.936516,0.976689,0.958815,0.102331,0.514047,0.988635,0.048224,0.129522
1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0.116504,0.503248,0.807974,0.225783,0.592678,0.436033,0.598435,0.506675,0.862466,0.479183,0.278588,0.470939,0.729264,0.624961,0.901829,0.812829,0.401039,0.450510,0.467991,0.450760,0.561798,0.799551,0.926962,0.943297,0.633036,0.013852,0.533221,0.600963,0.435323,0.230835,0.467376,0.617102,0.149082,1.0,0.573211,0.331577,0.819130,0.135787,0.516668,0.679413,0.522225,0.469625,0.271325,0.489737,0.764826,0.161923,0.628552,0.909069,0.989784,0.930195,0.638155,0.389996,0.867947,0.192102,0.451680,0.699709,0.198031,0.290148,0.550263,0.603532,0.441019,0.457699,0.226723,0.420190,0.495440,0.525674,0.367284,0.493723,0.394864,0.414946,0.051492,0.519341,0.611536,0.543484,0.539834,0.235869,0.614794,0.852725,0.521718,0.839149,0.449225,0.255697,0.303490,0.658337,0.305612,0.450865,0.293301,0.552003,0.507090,0.510608,0.339563,0.407132,0.128112,0.592731,0.396224,0.0,0.555675,0.159624,0.898993,0.209908,0.632088,0.413982,0.253659,0.267796,0.742450,0.956018,0.268475,0.869244,0.900040,0.872086,0.990271,0.177813,0.526318,0.483277,0.095336,0.360443,0.148238
2,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0.097081,0.507227,0.872440,0.269902,0.602877,0.432130,0.667726,0.887036,0.896088,0.479183,0.235460,0.380469,0.801084,0.619848,0.921414,0.853955,0.000000,0.459613,0.393383,0.431250,0.472504,0.911578,0.930051,0.947636,0.642306,0.013639,0.495870,0.599825,0.100512,0.413890,0.112541,0.527296,0.149

In [5]:
# we still have to do this on all the datasets, since we then use them for performance evaluation
X_full = df.iloc[:,:-1].values
y_full = df.loc[:,targ].values
print(len(X_full))
print(len(y_full))
print("Matrix of features", X_full, sep='\n')
print("--------------------------------------------------")
print("Target Variable", y_full, sep='\n')

2452
2452
Matrix of features
[[0.         1.         0.         ... 0.51404655 0.98863451 0.04822394]
 [0.         1.         0.         ... 0.48327729 0.09533563 0.36044311]
 [0.         1.         0.         ... 0.472904   0.76404125 0.18961851]
 ...
 [1.         0.         0.         ... 0.40617632 0.0595795  0.40162675]
 [1.         0.         0.         ... 0.49072939 0.09946237 0.33475944]
 [1.         0.         0.         ... 0.53567876 0.11938145 0.27496883]]
--------------------------------------------------
Target Variable
[0.12952186 0.14823844 0.1047106  ... 0.4377949  0.48584461 0.54089336]


In [6]:
# Label Encode the target variable
if targ in categoricals:
    label_encoder = LabelEncoder()
    encoded_y = label_encoder.fit_transform(y_full)
    label_encoder_name_mapping = dict(zip(label_encoder.classes_,
                                            label_encoder.transform(label_encoder.classes_)))
    print("Mapping of Label Encoded Classes", label_encoder_name_mapping, sep="\n")
    print("Label Encoded Target Variable", encoded_y, sep="\n")
else:
    pass

In [7]:
# splitting df_train in training and testing sets (only the TRAIN dataset)
X = X_full[:1828].copy()
print(len(X))
y = y_full[:1828].copy()
print(len(y))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=43)
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

1828
1828
1279
549
1279
549


## Feature Selection (Dimensionality Reduction)

### Univariate Feature Selection

In [8]:
# Decide K ----> Number of best features to select
n = 15
if targ in categoricals:
    selector = SelectKBest(k=n)

    X_train_sel = selector.fit(X_train, y_train)
    cols_idxs = selector.get_support(indices=True)
    cols_idxs = np.append(cols_idxs,[-1])
    df_UFS = df.iloc[:,cols_idxs]
    X_train_sel = selector.transform(X_train)
    print(df_UFS.shape)
    df_UFS.head()
    X_test_sel = selector.transform(X_test)

    decision_tree = DecisionTreeClassifier()
    param_grid = {
        'min_samples_leaf': [x for x in range(1,30)],
        'max_depth': [x for x in range(1,30)],
        'criterion':['gini','entropy']
    }
    grid_cv = GridSearchCV(decision_tree, param_grid, scoring="accuracy", n_jobs=-1, cv=5)
    grid_cv.fit(X_train_sel, y_train)
    print("Best Params", grid_cv.best_params_)
    print("Best CV Score", grid_cv.best_score_)
    print(f'Accuracy on Model 1 = {round(accuracy_score(y_test, grid_cv.predict(X_test_sel)), 10)}')

    decision_tree = DecisionTreeClassifier(min_samples_leaf = grid_cv.best_params_['min_samples_leaf'], max_depth = grid_cv.best_params_['max_depth'], criterion=grid_cv.best_params_['criterion'])
    decision_tree.fit(X_train_sel, y_train)
    y_pred = decision_tree.predict(X_test_sel)

    label_encoder = LabelEncoder()
    encoded_y_test = label_encoder.fit_transform(y_test)
    label_encoder_name_mapping = dict(zip(label_encoder.classes_,
                                            label_encoder.transform(label_encoder.classes_)))
    y_score = decision_tree.predict_proba(X_test_sel)

    print(f'Accuracy: {accuracy_score(y_test, y_pred)}' )
    print(f'F1-score: {f1_score(y_test, y_pred, average=None)}')
    print(classification_report(y_test, y_pred, digits=3))
else:
    selector = SelectKBest(k=n, score_func=f_regression)

    X_train_sel = selector.fit(X_train, y_train)
    cols_idxs = selector.get_support(indices=True)
    cols_idxs = np.append(cols_idxs,[-1])
    df_UFS = df.iloc[:,cols_idxs]
    X_train_sel = selector.transform(X_train)
    print(df_UFS.shape)
    df_UFS.head()
    X_test_sel = selector.transform(X_test)

    reg = LinearRegression()

    reg.fit(X_train, y_train)
    # evaluate the model
    yhat = reg.predict(X_test)
    # evaluate predictions
    mae = mean_absolute_error(y_test, yhat)
    print('MAE: %.3f' % mae)

(2452, 16)
MAE: 0.009


In [9]:
df_UFS.head()

,mfcc_max,mfcc_q25,mfcc_q50,mfcc_q99,zc_sum_w1,mfcc_std_w1,mfcc_q25_w1,zc_sum_w2,sc_sum_w2,zc_sum_w3,mfcc_q99_w3,sc_sum_w3,zc_sum_w4,mfcc_std_w4,mfcc_q25_w4,zc_sum
0,0.717587,0.896695,0.479183,0.752627,0.062556,0.933467,0.679413,0.224991,0.210342,0.224149,0.833184,0.307510,0.448805,0.866067,0.653604,0.129522
1,0.598435,0.862466,0.479183,0.729264,0.149082,0.819130,0.679413,0.198031,0.226723,0.235869,0.839149,0.255697,0.396224,0.898993,0.632088,0.148238
2,0.667726,0.896088,0.479183,0.801084,0.149221,0.821121,0.701664,0.130543,0.250507,0.219039,0.845064,0.222941,0.361620,0.903513,0.611607,0.104711
3,0.584642,0.892183,0.506330,0.731318,0.101420,0.800603,0.654612,0.154487,0.291251,0.211274,0.760320,0.227556,0.492813,0.785652,0.614810,0.138251
4,0.676841,0.852104,0.483519,0.747265,0.055005,0.903958,0.679413,0.196080,0.240252,0.257198,0.772112,0.207254,0.461552,0.840194,0.636019,0.122601


### Recursive Feature Elimination

In [10]:
# decide number of features to select
#n_features = [n for n in range(10,16)]
#for i in n_features:
#    selector = RFE(DecisionTreeClassifier(), n_features_to_select = i)
#    X_train_sel = selector.fit(X_train, y_train)
#    cols_idxs = selector.get_support(indices=True)
#    cols_idxs = np.append(cols_idxs,[-1])
#    df_RFE = df.iloc[:,cols_idxs]
#    X_train_sel = selector.transform(X_train)
#    print(df_RFE.shape)
#
#    X_test_sel = selector.transform(X_test)
#
#    decision_tree = DecisionTreeClassifier(min_samples_leaf=6) #random_state=78)
#    decision_tree.fit(X_train_sel, y_train)
#
#    y_pred = decision_tree.predict(X_test_sel)
#
#    label_encoder = LabelEncoder()
#    encoded_y_test = label_encoder.fit_transform(y_test)
#    label_encoder_name_mapping = dict(zip(label_encoder.classes_,
#                                            label_encoder.transform(label_encoder.classes_)))
#    y_score = decision_tree.predict_proba(X_test_sel)
#
#    print(f'Accuracy: {accuracy_score(y_test, y_pred)}' )
#    print(f'F1-score: {f1_score(y_test, y_pred, average=None)}')
#    print(classification_report(y_test, y_pred, digits=3))

In [11]:
if targ in categoricals:   
    selector = RFE(DecisionTreeClassifier(), n_features_to_select = 15)
    X_train_sel = selector.fit(X_train, y_train)
    cols_idxs = selector.get_support(indices=True)
    cols_idxs = np.append(cols_idxs,[-1])
    df_RFE = df.iloc[:,cols_idxs]
    X_train_sel = selector.transform(X_train)
    print(df_RFE.shape)

    X_test_sel = selector.transform(X_test)

    decision_tree = DecisionTreeClassifier(min_samples_leaf=6) #random_state=78)
    decision_tree.fit(X_train_sel, y_train)

    y_pred = decision_tree.predict(X_test_sel)

    label_encoder = LabelEncoder()
    encoded_y_test = label_encoder.fit_transform(y_test)
    label_encoder_name_mapping = dict(zip(label_encoder.classes_,
                                            label_encoder.transform(label_encoder.classes_)))

    y_score = decision_tree.predict_proba(X_test_sel)

    print(f'Accuracy: {accuracy_score(y_test, y_pred)}' )
    print(f'F1-score: {f1_score(y_test, y_pred, average=None)}')
    print(classification_report(y_test, y_pred, digits=3))
else:
    pass

In [12]:
try:
    df_RFE.head()
except:
    pass

In [13]:
df_UFS.to_csv('data_UFS_advReg.csv', index=False)
#df_RFE.to_csv('data_RFE_advClass.csv', index=False)